In [1]:
import pandas as pd
import numpy as np
df = pd.read_csv("AAA.csv")
df.head()

,Date,AAPL,GOOGL,AMZN
0,11/20/2014,106.867630,543.760010,330.540009
1,11/21/2014,107.014664,545.890015,332.630005
2,11/24/2014,108.999298,547.479980,335.640015
3,11/25/2014,108.052910,549.229980,335.040009
4,11/26/2014,109.339249,547.729980,333.570007


In [2]:
dt = 1/252
prices = df
n0 = len(prices)
prices = prices.drop(columns='Date')
log_prices = prices.apply(lambda x: np.log(x))
log_returns = log_prices.apply(lambda x: np.diff(x)) 

v = (log_returns.mean()/dt).values
Sigma = log_returns.cov()/dt

print(f"v: {v}\nSigma (covariance matrix): \n{Sigma}")

v: [0.18054363 0.17488585 0.33334629]
Sigma (covariance matrix): 
           AAPL     GOOGL      AMZN
AAPL   0.062493  0.031321  0.036269
GOOGL  0.031321  0.056491  0.044934
AMZN   0.036269  0.044934  0.084675


In [14]:
np.random.seed(11)
def sim_multi_GBM(S0, v, Sigma, delta_t, T):
    """GBM simulation of multiple asset paths

    Args:
        S0 (vector of start prices): vector of starting prices
        v (array): mu/dt
        Sigma (Series): covariance matrix
        delta_t (float): delta between each time step
        T (time): maturity time
    """

    m = int(T/delta_t)
    p = len(S0)  # number of assets
    S = [] 
    S.append(S0)

    z = np.random.multivariate_normal(mean=v*delta_t, cov=Sigma.to_numpy()*delta_t, size=m)
    for step in range(1, m):
        S.append(np.exp(np.log(S[step-1]) + z[step-1]))
    return np.array(S)

In [43]:
# simulate stock path prices
Nsim = 10
T = 1
dt = 1/252
S0 = prices.to_numpy()[-1]
m = int(T/dt)

# initialize ndarray to record stock prices
S2 = np.ndarray(shape=(Nsim, m))
S3 = np.ndarray(shape=(Nsim, m))

# 
S1 = []
S2 = []
S3 = []
for i in range(Nsim):
    S = sim_multi_GBM(S0, v, Sigma, dt, T)
    S1.append(S[:, [0]].reshape(-1))
    S2.append(S[:, [1]].reshape(-1))
    S3.append(S[:, [2]].reshape(-1))

In [46]:
import plotly.express as px
def plot_simulations(price_hist, sim_paths, stock=0):
    """plots simulations for chosen stock

    Args:
        price_hist (df): df of historical prices
        sim_paths (list): simulated price paths
        stock (int, optional): index of stock. Defaults to 0.
    """
    # df of historical prices
    price_hist = price_hist.to_numpy()[:, [0]].reshape(-1)
    price_hist = np.vstack([price_hist]*Nsim)
    price_hist = pd.DataFrame(price_hist).transpose()

    sim_paths = pd.DataFrame(S1).transpose()
    sim_paths = pd.concat((price_hist, sim_paths))
    sim_paths = sim_paths.reset_index(drop=True)
    fig = px.line(sim_paths)
    return fig

In [47]:
plot_simulations(prices, S1)